## Exploring Permissions

What permissions can we grant? How are effective determined?

In [ ]:
/*  local server principals 
    note roles are simply principals */
SELECT principal_id, name, type_desc, owning_principal_id, is_fixed_role
FROM sys.server_principals
WHERE name NOT LIKE '##%'
    AND name NOT LIKE 'NT%'
    AND name NOT LIKE 'lappytron%'
ORDER BY is_fixed_role, name;

/*  server_role_members is a lookup table to associate a server_principal 
    with server roles */
SELECT drm.role_principal_id, dbroles.name AS RoleName, drm.member_principal_id, ISNULL (roleMember.name, 'No members') AS ServerPrincipal   
FROM sys.server_role_members AS drm
    RIGHT OUTER JOIN sys.server_principals AS dbroles ON drm.role_principal_id = dbroles.principal_id  
 LEFT OUTER JOIN sys.server_principals AS roleMember  
   ON drm.member_principal_id = roleMember.principal_id  
WHERE dbroles.type = 'R'
    AND roleMember.name NOT LIKE '##%'
    AND roleMember.name NOT LIKE 'NT%'
    AND roleMember.name NOT LIKE 'lappytron%'
ORDER BY dbroles.name, roleMember.name; 


In [ ]:
/*  granting specific permissions */

GRANT ALTER ANY LOGIN TO accessTest AS sa;

EXECUTE AS LOGIN = 'accessTest'

CREATE LOGIN myNewLogin WITH PASSWORD = 'whocares';

REVERT

REVOKE ALTER ANY LOGIN TO accessTest AS sa;


In [ ]:
/*  permissions to permission can be given */
--DROP LOGIN myNewLogin;
--GO

GRANT ALTER ANY LOGIN TO accessTest WITH GRANT OPTION AS sa;
GRANT IMPERSONATE ANY LOGIN TO accessTest;

EXECUTE AS LOGIN = 'accessTest'

    GRANT ALTER ANY LOGIN TO secondTester AS accessTest;

BEGIN
    EXECUTE AS LOGIN = 'secondTester'

    CREATE LOGIN myOtherNewLogin WITH PASSWORD = 'stilldontcare'

    REVERT
END

REVERT

In [ ]:
/*  works the same in databases */
USE WideWorldImporters;

SELECT [principal_id], [name], [type_desc], [owning_principal_id], [is_fixed_role]
FROM [sys].[database_principals]
WHERE ([principal_id] IN (0,1,2,3,4,14,15)) 
    OR (type_desc = 'DATABASE_ROLE' AND [name] LIKE 'db%')
ORDER BY [is_fixed_role], [name];

SELECT [drm].[role_principal_id], [dbroles].[name] AS [Role Name], [drm].[member_principal_id], ISNULL ([roleMember].[name], 'No members') AS [Server Principal]   
FROM [sys].[database_role_members] AS [drm]
    RIGHT OUTER JOIN [sys].[database_principals] AS [dbroles] ON [drm].[role_principal_id] = [dbroles].[principal_id]  
 LEFT OUTER JOIN [sys].[database_principals] AS [roleMember]  
   ON [drm].[member_principal_id] = [roleMember].[principal_id]  
WHERE [dbroles].[type] = 'R'
    AND [dbroles].[name] LIKE 'db%'
ORDER BY [dbroles].[name], [roleMember].[name];

In [ ]:
/*  granting permissions */
USE WideWorldImporters;
GO

CREATE USER accessTest FOR LOGIN accessTest;

EXECUTE AS LOGIN = 'accessTest';

BEGIN TRY
    SELECT TOP (10) *
    FROM Sales.Orders;
END TRY
BEGIN CATCH
    PRINT 'Nae Luck!';
END CATCH

REVERT;

GRANT SELECT ON Sales.Orders TO accessTest;

EXECUTE AS LOGIN = 'accessTest';

BEGIN TRY
    SELECT TOP (10) *
    FROM Sales.Orders;
END TRY
BEGIN CATCH
    PRINT 'Still Nae Luck!';
END CATCH

REVERT;

REVOKE SELECT ON Sales.ORders TO accessTest;


In [ ]:
/*  sharing permissions */
USE WideWorldImporters;
GO

GRANT SELECT ON Sales.Orders TO accessTest WITH GRANT OPTION;

EXECUTE AS LOGIN = 'accessTest';

    EXECUTE AS LOGIN = 'secondTester';

    BEGIN TRY
        SELECT TOP (10) *
        FROM Sales.Orders;
    END TRY
    BEGIN CATCH
        PRINT 'Nae Luck for the second tester!';
    END CATCH

    REVERT

    GRANT SELECT ON Sales.Orders TO secondTester;

    EXECUTE AS LOGIN = 'secondTester';

    BEGIN TRY
        SELECT TOP (10) *
        FROM Sales.Orders;
    END TRY
    BEGIN CATCH
        PRINT 'Still Nae Luck for the second tester!';
    END CATCH

    REVERT;

REVERT;


In [ ]:
/* permissions are chained! */

REVOKE SELECT ON Sales.Orders FROM accessTest;

In [ ]:
/* Properly Removed! */
USE WideWorldImporters
GO

REVOKE SELECT ON Sales.Orders TO accessTest CASCADE;

/* WARNING! REVOKES ALL GRANTS _AND_DENYS! */
GO

EXECUTE AS LOGIN = 'accessTest';

BEGIN TRY
    SELECT TOP (10) *
    FROM Sales.Orders;
END TRY
BEGIN CATCH
    PRINT 'Nae Luck!';
END CATCH

REVERT;

EXECUTE AS LOGIN = 'secondTester';

BEGIN TRY
    SELECT TOP (10) *
    FROM Sales.Orders;
END TRY
BEGIN CATCH
    PRINT 'Nae Luck for the second tester!';
END CATCH

REVERT

In [ ]:
/*  How effective are denies? */
USE WideWorldImporters;
GO

ALTER ROLE db_owner ADD MEMBER accessTest;
GO

DENY SELECT ON Sales.Orders TO accessTest;

EXECUTE AS LOGIN = 'accessTest'

BEGIN TRY
    SELECT TOP (10) *
    FROM Sales.Orders;
END TRY
BEGIN CATCH
    PRINT 'NO SALES ORDERS FOR YOU!'
END CATCH

SELECT TOP (5) *
FROM Sales.Invoices;

REVERT


In [ ]:
/*  Does it hold up for sysadmins? */
ALTER SERVER ROLE sysadmin ADD MEMBER accessTest;

USE WideWorldImporters;
GO

EXECUTE AS LOGIN = 'accessTest'

BEGIN TRY
    SELECT TOP (10) *
    FROM Sales.Orders;
END TRY
BEGIN CATCH
    PRINT 'NO SALES ORDERS FOR YOU!'
END CATCH

SELECT TOP (5) *
FROM Sales.Invoices;

REVERT

ALTER SERVER ROLE  sysadmin DROP MEMBER accessTest;

In [ ]:
/*  object owner is like a mini-sa permission */
USE WideWorldImporters;
GO

ALTER ROLE db_owner ADD MEMBER accessTest;
GO

EXECUTE AS LOGIN = 'accessTest'

BEGIN TRY
    SELECT TOP (10) *
    FROM Sales.Orders;
END TRY
BEGIN CATCH
    PRINT 'NO SALES ORDERS FOR YOU!'
END CATCH

SELECT TOP (5) *
FROM Sales.Invoices;

REVERT

SELECT [drm].[role_principal_id], [dbroles].[name] AS [Role Name], [drm].[member_principal_id], ISNULL ([roleMember].[name], 'No members') AS [Server Principal]   
FROM [sys].[database_role_members] AS [drm]
    RIGHT OUTER JOIN [sys].[database_principals] AS [dbroles] ON [drm].[role_principal_id] = [dbroles].[principal_id]  
 LEFT OUTER JOIN [sys].[database_principals] AS [roleMember]  
   ON [drm].[member_principal_id] = [roleMember].[principal_id]  
WHERE [dbroles].[type] = 'R'
    AND [dbroles].[name] LIKE 'db%'
ORDER BY [dbroles].[name], [roleMember].[name];

ALTER ROLE db_owner DROP MEMBER accessTest;


## Server Level Roles

Why would we create a custom server level role?

* allow applications to create/drop their own databases (but **only** their own databases). Unfortunately db_creator grants permissions ALTER ANY DATABASE and CREATE ANY DATABASE which would allow an application to drop _any_ database on the instance.
* allow developers access to diagnostic information without exposing other permissions.



In [ ]:

/* Our application needs db_creator to create databases as needed... */
USE master;
GO

CREATE LOGIN testdbCreator WITH PASSWORD = 'soSecure';
ALTER SERVER ROLE dbcreator ADD MEMBER testdbCreator;

In [ ]:

/* Works great! */

GO

EXECUTE AS LOGIN = 'testdbCreator';

CREATE DATABASE safeToDrop;

--DROP DATABASE safeToDrop;

REVERT;

In [ ]:

/*  But wait!
    ALTER ANY DATABASE permissions are too much! */
EXECUTE AS LOGIN = 'testdbCreator';

BEGIN TRY
    CREATE DATABASE safeToDrop;
    PRINT 'Created database safeToDrop...'
    
    DROP DATABASE safeToDrop;
    PRINT 'Dropped database safeToDrop...'

    /* this should always fail! */
    DROP DATABASE neverDrop;
    PRINT 'Oops: Dropped database neverDrop!'

END TRY
BEGIN CATCH
    PRINT 'neverDrop DB should be safe.'
END CATCH

REVERT;

In [ ]:

/*  Reset 
    NeverDrop and create a custom role! */
CREATE DATABASE neverDrop;
GO

CREATE SERVER ROLE [crole_CreateDB];
GO

GRANT CREATE ANY DATABASE TO [crole_CreateDB];
GO

ALTER SERVER ROLE dbcreator DROP MEMBER testdbCreator;
GO
ALTER SERVER ROLE crole_CreateDB ADD MEMBER testdbCreator;
GO

EXECUTE AS LOGIN = 'testdbCreator';
CREATE DATABASE safeToDrop;
REVERT;

In [ ]:

/*  Specific Permission is Specific:
    CREATE ANY DATABASE is just right...*/
BEGIN TRY
    EXECUTE AS LOGIN = 'testdbCreator';

    DROP DATABASE safeToDrop;
    PRINT 'Dropped database safeToDrop...'

    /* this should always fail! */
    DROP DATABASE neverDrop;
    PRINT 'Oops: Dropped database neverDrop!'

END TRY
BEGIN CATCH
    PRINT 'neverDrop DB should be safe.'
END CATCH

REVERT;

In [ ]:

/* Now Chad needs SA!?? */
CREATE LOGIN Chad WITH PASSWORD = 'soSecure2';
ALTER SERVER ROLE [serveradmin] ADD MEMBER Chad;
GO

EXECUTE AS LOGIN = 'Chad';
BEGIN TRY

    SELECT TOP (10) wait_type, wait_time_ms 
    FROM sys.dm_os_wait_stats ORDER BY wait_time_ms DESC
    PRINT 'Chad has retrieved useful wait stats from the server!'

    EXEC sp_configure 'show advanced options', 1
    PRINT 'Chad has access advanced server options!'
    RECONFIGURE
    
    EXEC sp_configure 'max degree of parallelism', 24
    PRINT 'Chad has created a parallelism problem!'
    RECONFIGURE WITH OVERRIDE

END TRY
BEGIN CATCH
    PRINT 'You don''t have permission for that!'
END CATCH

REVERT;
GO


In [ ]:

/* Cleaning up after Chad... */
USE master;

ALTER SERVER ROLE [serveradmin] DROP MEMBER Chad;

EXEC sp_configure 'Show advanced Options', 1;
RECONFIGURE

EXEC sp_configure 'max degree of parallelism', 2;
EXEC sp_configure 'show advanced options', 0;
RECONFIGURE;
GO

CREATE SERVER ROLE crole_Troubleshooting;
GO

GRANT VIEW SERVER STATE TO crole_Troubleshooting;
-- maybe Chad wants to use SQL Profiler (gross!)
GRANT ALTER TRACE TO crole_Troubleshooting;
-- team Extended Events all the way! 😎
GRANT ALTER ANY EVENT SESSION TO crole_Troubleshooting;

ALTER SERVER ROLE crole_Troubleshooting ADD MEMBER Chad;
GO

In [ ]:

/*  Chad can see what's wrong but is not empowerd 
    to be "creative" with his solutions. */
EXECUTE AS LOGIN = 'Chad';
BEGIN TRY
    SELECT TOP (10) wait_type, wait_time_ms 
    FROM sys.dm_os_wait_stats ORDER BY wait_time_ms DESC
    PRINT 'Chad has retrieved useful wait stats from the server!'

    EXEC sp_configure 'show advanced options', 1
    PRINT 'Chad has access advanced server options!'
    RECONFIGURE
    
    EXEC sp_configure 'max degree of parallelism', 24
    PRINT 'Chad has created a parallelism problem!'
    RECONFIGURE WITH OVERRIDE

END TRY
BEGIN CATCH
    PRINT 'You don''t have permission for that!'
END CATCH

REVERT;
GO

## Database Level Roles

Database roles allow us to control access to data and procedures in each specific database including:

* Allow or deny access to data,
* allow write access to data,
* create or delete database objects.

In [ ]:

/* Chad needs to do some research on the data though...*/
ALTER SERVER ROLE crole_Troubleshooting DROP MEMBER [Chad]
GO

EXECUTE AS LOGIN = 'Chad';
BEGIN TRY

    USE WideWorldImporters;

    SELECT dm.DeliveryMethodName
        , dm.validFrom
        , dm.ValidTo
        , p.preferredName
    FROM application.People p JOIN application.DeliveryMethods dm ON dm.LastEditedBy = p.PersonID;

END TRY
BEGIN CATCH
    PRINT ERROR_MESSAGE();
END CATCH

REVERT;
GO

In [ ]:

/*  Just becuase we grant him read doesn't 
    mean he can get what he needs though! */

USE [WideWorldImporters]
GO

CREATE USER [Chad] FOR LOGIN [Chad]
GO
ALTER ROLE [db_datareader] ADD MEMBER [Chad]
GO


EXECUTE AS LOGIN = 'Chad';
BEGIN TRY

    USE WideWorldImporters;

    SELECT p.PurchaseOrderID,
       p.OrderDate,
       s.SupplierName,
       pc.FullName,
       pc.PreferredName
    FROM Purchasing.PurchaseOrders p
    JOIN Purchasing.Suppliers s
        ON p.SupplierID = s.SupplierID
    JOIN Application.People AS pc
        ON pc.PersonID = s.PrimaryContactPersonID
    WHERE p.OrderDate >= '2010-01-01'
      AND p.OrderDate < '2014-01-01'
    ORDER BY p.OrderDate DESC;

    EXEC Integration.GetOrderUpdates @LastCutoff = '2010-01-01',
                                 @NewCutoff = '2014-01-01';

END TRY
BEGIN CATCH
    PRINT ERROR_MESSAGE();
END CATCH

REVERT;
GO

In [ ]:

/* Executing procedures and functions have their own permission */
USE WideWorldImporters;
GO
GRANT EXECUTE ON OBJECT::Integration.GetOrderUpdates TO Chad;
GO

EXECUTE AS LOGIN = 'Chad';
BEGIN TRY

    USE WideWorldImporters;

    EXEC Integration.GetOrderUpdates @LastCutoff = '2010-01-01',
                                 @NewCutoff = '2014-01-01';

END TRY
BEGIN CATCH
    PRINT ERROR_MESSAGE();
END CATCH

REVERT;
GO

In [ ]:

/*  Can't troubleshoot without an execution plan.
    Run Chad's Query FIRST */
USE WideWorldImporters;
GO

GRANT SHOWPLAN TO Chad;


## Implementing Database Roles to Simplify Data Access
* Just like server level roles but for databases!

In [ ]:

/* Way more columns than report writers need! */
USE AdvWorksDW;
GO

SELECT TOP (100) [AccountKey]
      ,[ParentAccountKey]
      ,[AccountCodeAlternateKey]
      ,[ParentAccountCodeAlternateKey]
      ,[AccountDescription]
      ,[AccountType]
      ,[Operator]
      ,[CustomMembers]
      ,[ValueType]
      ,[CustomMemberOptions]
      ,[ETL_NATRL_KEY_TXT]
      ,[ETL_VERS_START_DTM]
      ,[ETL_VERS_END_DTM]
      ,[ETL_VERS_CURR_FLAG]
      ,[ETL_JOB_SEQ_NUM]
      ,[ETL_SRC_SYS_NM]
      ,[ETL_SRC_TABLE_NM]
      ,[ETL_IUD_CD]
      ,[ETL_SNAPSHOT_DTM]
      ,[ETL_INS_DTM]
      ,[ETL_INS_JOB_NM]
      ,[ETL_UPD_DTM]
      ,[ETL_UPD_JOB_NM]
      ,[ETL_UPD_ROWVERSION]
      ,[ETL_DEL_DTM]
      ,[ETL_CHG_CNTRL_CHECKSUM]
      ,[ETL_CHG_CNTRL_2_CHECKSUM]
      ,[ETL_CHG_CNTRL_3_CHECKSUM]
  FROM [AdvWorksDW].[dbo].[DimAccount]

In [ ]:

/* simplified for reporting */
USE AdvWorksDWViews;
GO
/*

 <-- let's expand and look at views in my alternate univers database

*/
CREATE USER [Chad] FOR LOGIN [Chad]
GO

GRANT SELECT ON SCHEMA::rpt to Chad;
GO

EXECUTE AS LOGIN = 'Chad';

    SELECT AccountKey
        , AccountType
        , AccountDescription
        , AccountType
    --    , ETL_SRC_TABLE
    FROM rpt.dimAccount

    SELECT *
    FROM dbo.dimAccount

REVERT;

In [ ]:

/* restricted information is easier to contain! */
/*

 <-- let's expand and look at views in my alternate univers database

*/
USE AdvWorksDWViews;
GO

EXECUTE AS LOGIN = 'Chad';

BEGIN TRY
    SELECT *
    FROM rpt.dimEmployee

    SELECT *
    FROM secureRpt.dimEmployee
END TRY
BEGIN CATCH
    PRINT ERROR_MESSAGE();
END CATCH

REVERT;

In [ ]:

/* create our HR user */
USE master;
CREATE LOGIN SusanInHR WITH PASSWORD = 'soSecure3';

USE AdvWorksDWViews;
GO
CREATE USER [SusanInHR] FOR LOGIN [SusanInHR]
GO

GRANT SELECT ON SCHEMA::rpt to SusanInHR;
GO

GRANT SELECT ON SCHEMA::secureRpt to SusanInHR;
GO

In [ ]:

/* restricted information is available to those with access only! */
USE AdvWorksDWViews;
GO

EXECUTE AS LOGIN = 'SusanInHr'
    SELECT *
    FROM rpt.dimEmployee

    SELECT *
    FROM secureRpt.dimEmployee
REVERT;

In [ ]:

/* Clean up user permissions and make this much more managable */

USE [AdvWorksDWViews];
GO
REVOKE SELECT ON SCHEMA::[rpt] TO [Chad];
GO

REVOKE SELECT ON SCHEMA::[rpt] TO [SusanInHR];
GO

REVOKE SELECT ON SCHEMA::[secureRpt] TO [SusanInHR];
GO

CREATE ROLE [crole_StandardReports] AUTHORIZATION [dbo];
GO
GRANT SELECT ON SCHEMA::[rpt] TO [crole_StandardReports];
GO

CREATE ROLE [crole_SensitiveReports] AUTHORIZATION [dbo];
GO
GRANT SELECT ON SCHEMA::[secureRpt] TO [crole_SensitiveReports];
GO

ALTER ROLE [crole_SensitiveReports] ADD MEMBER [SusanInHR];
GO

ALTER ROLE [crole_StandardReports] ADD MEMBER [chad];
GO

ALTER ROLE [crole_StandardReports] ADD MEMBER [SusanInHR]
GO

In [ ]:

/*  In the realworld Chad and Susan would be 
    nested inside the appropraite security groups! */
USE AdvWorksDWViews;
GO

EXECUTE AS LOGIN = 'Chad';

    SELECT 'Chad' AS uName, *
    FROM rpt.dimEmployee

    SELECT 'Chad' as uName, *
    FROM secureRpt.dimEmployee
    GO

REVERT;

EXECUTE AS LOGIN = 'SusanInHr'
    SELECT 'Susan' AS uName, *
    FROM rpt.dimEmployee

    SELECT 'Susan' AS uName, *
    FROM secureRpt.dimEmployee
    GO

REVERT;